In [67]:
import pandas as pd


**Задание 1**

Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце 'region' пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим 'undefined'.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

geo_data = {

    'Центр': ['москва', 'тула', 'ярославль'],

    'Северо-Запад': ['петербург', 'псков', 'мурманск'],

    'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']

}

Результат классификации запишите в отдельный столбец region.

In [68]:
geo_data = {

'Центр': ['москва', 'тула', 'ярославль'],

'Северо-Запад': ['петербург', 'псков', 'мурманск'],

'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

In [69]:
data_key = pd.read_csv('keywords.csv')

In [70]:
def add_region(row):
    for i in geo_data['Центр']:
        if i in row['keyword']:
            return 'Центр'
    for i in geo_data['Северо-Запад']:
        if i in row['keyword']:
            return 'Северо-Запад'
    for i in geo_data['Дальний Восток']:
        if i in row['keyword']:
            return 'Дальний Восток'
        return 'undefined'

In [71]:
data_key['region'] = data_key.apply(add_region, axis=1)

In [72]:
data_key['region'].value_counts()

undefined         99304
Центр               387
Северо-Запад        266
Дальний Восток       43
Name: region, dtype: int64

**Задание 2**

Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
    - оценка 2 и меньше - низкий рейтинг
    - оценка 4 и меньше - средний рейтинг
    - оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class

In [73]:
movies = pd.read_csv ('movies.csv')
ratings = pd.read_csv ('ratings.csv')
movies_ratings = ratings.merge (movies)

In [74]:
def class_movie (row):
    if row ['rating'] <= 2:
        return 'низкий рейтинг'
    if row ['rating'] <= 4:
        return 'средний рейтинг'
    else:
        return 'высокий рейтинг'

In [75]:
movies_ratings ['class'] = movies_ratings.apply (class_movie, axis = 1)
movies_ratings.head()

,userId,movieId,rating,timestamp,title,genres,class
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,средний рейтинг
1,5,1,4.0,847434962,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,средний рейтинг
2,7,1,4.5,1106635946,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,высокий рейтинг
3,15,1,2.5,1510577970,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,средний рейтинг
4,17,1,4.5,1305696483,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,высокий рейтинг


**Задание 3**

Посчитайте среднее значение Lifetime киноманов (пользователи, которые поставили 100 и более рейтингов). Под Lifetime понимается разница между максимальным и минимальным значением timestamp для каждого пользователя. Ответ дайте в днях.

In [12]:
ratings = pd.read_csv ('ratings.csv').groupby ('userId').agg ((min, max))
ratings ['diff'] = ratings ['timestamp'] ['max'] - ratings ['timestamp'] ['min']
ratings ['diff'].mean () / 24 / 3600

234.29512251442014

**Задание 4**

Есть мнение, что "раньше снимали настоящее кино, не то что сейчас". Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из материалов занятия. Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

1. В переменную years запишите список из всех годов с 1950 по 2010.

In [25]:
movies = pd.read_csv ('movies.csv')
ratings = pd.read_csv ('ratings.csv')
years = [x for x in range (1950, 2011)]

2. Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:
    - для каждой строки пройдите по всем годам списка years
    - если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
    - если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год

In [31]:
def production_year(row):
    for x in years:
        if str(x) in row['title']:
            return str(x)
    return '1900'

3. Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец 'year'

In [33]:
movies['years'] = movies.apply(production_year, axis=1)

In [35]:
movies.head()

,movieId,title,genres,years
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men (1995),Comedy|Romance,1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II (1995),Comedy,1995


4. Посчитайте средний рейтинг всех фильмов для каждого значения столбца 'year' и отсортируйте результат по убыванию рейтинга

In [50]:
mean_rating = ratings.merge(movies)
mean_rating.groupby('years').agg(['mean'])['rating'].sort_values('mean', ascending=False).head()

,mean
years,
1957,4.039535
1954,4.009191
1962,3.969466
1952,3.953125
1972,3.944293


In [ ]:
#Ответ: Утверждение верно